会在训练集目录下多两个文件夹：
abnormImages ： 把异常图片拿出来好做处理
abnormImagesLabel ：把异常图片打上标签

In [19]:
import os
import shutil
import warnings
import json
from PIL import Image, ImageDraw
from PIL import ImageFont
warnings.filterwarnings('ignore')

In [20]:
# 改一下 BASE_PATH 运行三次
BASE_PATH = "../../../train/01/"

IMG_TRAIN_PATH = os.path.join(BASE_PATH, "Images")
XML_PATH = os.path.join(BASE_PATH, "Annotations")
ABNORM_IMG_PATH = os.path.join(BASE_PATH, "abnormImages")
ABNORM_LABEL_PATH = os.path.join(BASE_PATH, "abnormImagesLabel")

IMG_TEST_PATH = '../../../test/Images/'
IMAGE_TYPE = '.jpg'

# 1. 将三类10张异常图片拿出来放到一个文件夹当中

In [21]:
# os.walk方法用于通过在目录树中游走输出在目录中的文件名
def getOutFileList(path):
    for root, dirs, files in os.walk(path):
        return files
file_name = getOutFileList(XML_PATH)
image_name_lists = getOutFileList(IMG_TRAIN_PATH)

In [22]:
# 异常图片索引
def abnormal_index(file_name):
    abnormal_args = []
    for name in file_name:
        abnormal_args.append(int(os.path.splitext(name)[0].split('_')[1]))
    return abnormal_args

abnormal_args = abnormal_index(file_name)
abnormal_args

[1, 2, 21, 34, 37, 44, 52, 66, 71, 79]

In [23]:
def create_abnormal_lists(image_name_lists, abnormal_args, old_file_path ,outlier_file):
    '''
    创建abnormalImage的文件夹,里面放复制过来的异常图片
    :param image_name_lists: 全部图片的数组, list格式
    :param abnormal_args: 异常图片的数组索引, list格式
    :param outlier_file: 输出图片的位置
    :param old_file_path: 需要复制图片的位置
    '''
    # 判断路径是否存在
    isExists = os.path.exists(outlier_file)
    file_img = old_file_path
    # 判断结果
    if not isExists:
        # 如果不存在则创建目录
        # 创建目录操作函数
        os.makedirs(outlier_file)
        print(outlier_file + ' 创建成功')

        for i in image_name_lists:
            num = int(os.path.splitext(i)[0].split('_')[1])
            if num in abnormal_args:
                old_file_path = os.path.join(file_img,i)
                new_file_path = os.path.join(outlier_file,i)
                shutil.copy(old_file_path, new_file_path)
        return True  
    else:
        # 如果目录存在则不创建，并提示目录已存在
        print(outlier_file + ' 目录已存在')
        return False

In [24]:
create_abnormal_lists(image_name_lists,abnormal_args,IMG_TRAIN_PATH,ABNORM_IMG_PATH)

../../../train/01/abnormImages 创建成功


True

# 2.  给现有的异常图片用bbox打上标签

In [25]:
def pil_draw(img_path,abnormal_file,outlier_file,pic_type):
    # 读取json文件
    with open(os.path.join(img_path,'TRAIN_objects.json'),'r') as load_f:
        load_dict = json.load(load_f)
    # print(load_dict)
    for item in load_dict:
        # print(item)
        # 1.读取图片
        im = Image.open(os.path.join(abnormal_file,item['name'] + pic_type))
        # 2.获取标签 box_list
        draw = ImageDraw.Draw(im)
        for box_list,label in zip(item['boxes'],item['labels']):
            # print(box_list)
            draw.rectangle([box_list[0], box_list[1], box_list[2], box_list[3]],outline='red',width=1)  # 画bbox
            font = ImageFont.truetype("consola.ttf", 12, encoding="unic" )  # 设置字体
            draw.text((box_list[0]-15, box_list[1]-15),label, 'green', font)    # 写label
        del draw
        # 3.保存图片
        isExists = os.path.exists(outlier_file)
        if not isExists:
            os.makedirs(outlier_file)   # 创建文件路径
        else:
            im.save(os.path.join(outlier_file, item['name'] + pic_type))   # 保存文件
    print('保存成功')

In [26]:
outlier_file = ABNORM_LABEL_PATH

# 第二类数据有点奇怪，给的都是绿色，出来不是
pil_draw(BASE_PATH,ABNORM_IMG_PATH,outlier_file,IMAGE_TYPE)

保存成功
